# Generate training and prediction data from insitu and gridded datasets


## Load modules

In [1]:
from datacube.utils.dask import start_local_dask

import sys
sys.path.append('/g/data/os22/chad_tmp/NEE_modelling/')
from collect_training_data import extract_ec_gridded_data

In [2]:
client = start_local_dask(mem_safety_margin='2Gb')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 1
Total threads: 16,Total memory: 44.92 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34153,Workers: 1
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 44.92 GiB
Comm: tcp://127.0.0.1:45385,Total threads: 16
Dashboard: /proxy/44985/status,Memory: 44.92 GiB
Nanny: tcp://127.0.0.1:44875,


## Collect training data from EC towers, and gridded data at pixel location of EC tower

This data is used for training the models.

'Yanco/2021_v1/L6/default/Yanco_L6_20130101_20210724_Monthly.nc', soil site

'ozflux/Collie/2021_v1/L5/default/Collie_L5_20170804_20191111.nc', COLLIE SITE AT Level 5

In [3]:
suffixes = [
    'CumberlandPlain/2022_v1/L6/default/CumberlandPlain_L6_20140101_20220101_Monthly.nc',
    'Tumbarumba/2021_v1/L6/default/Tumbarumba_L6_20020107_20191231_Monthly.nc',
    'Whroo/2021_v1/L6/default/Whroo_L6_20111201_20210724_Monthly.nc',
    'WombatStateForest/2022_v1/L6/default/WombatStateForest_L6_20100120_20210529_Monthly.nc',
    'WallabyCreek/2022_v1/L6/default/WallabyCreek_L6_20050825_20130409_Monthly.nc',
    'RobsonCreek/2022_v1/L6/default/RobsonCreek_L6_20130801_20211218_Monthly.nc',
    'CapeTribulation/2022_v1/L6/default/CapeTribulation_L6_20100101_20181102_Monthly.nc',
    'AliceSpringsMulga/2022_v1/L6/default/AliceSpringsMulga_L6_20100903_20211231_Monthly.nc',
    'CalperumChowilla/2022_v1/L6/default/Calperum_L6_20100730_20220216_Monthly.nc',
    'DryRiver/2022_v1/L6/default/DryRiver_L6_20091025_20220218_Monthly.nc',
    'Litchfield/2021_v1/L6/default/Litchfield_L6_20150623_20210725_Monthly.nc',
    'Gingin/2021_v1/L6/default/Gingin_L6_20111013_20201231_Monthly.nc',
    'TiTreeEast/2022_v1/L6/default/TiTreeEast_L6_20120718_20220117_Monthly.nc',
    'SturtPlains/2021_v1/L6/default/SturtPlains_L6_20080828_20210724_Monthly.nc',
    'RiggsCreek/2022_v1/L6/default/RiggsCreek_L6_20110101_20170712_Monthly.nc',
    'DalyPasture/2022_v1/L6/default/DalyPasture_L6_20080101_20130908_Monthly.nc',
    'Otway/2021_v1/L6/default/Otway_L6_20070811_20110101_Monthly.nc',
]

i=0
sites = []
for suffix in suffixes:
    print(" {:02}/{:02}\r".format(i + 1, len(suffixes)), end="")
    xx = extract_ec_gridded_data(suffix)
    sites.append(xx)
    i+=1
    

## Collate gridded data for predictions

This data is used to predict on, results of this are stored here `/g/data/os22/chad_tmp/NEE_modelling/results/predicton_data/prediction_data_YYYY.nc`

In [ ]:
import sys
sys.path.append('/g/data/os22/chad_tmp/NEE_modelling/')
from collect_prediction_data import collect_prediction_data

In [ ]:
t1, t2='2002-10','2021'

In [ ]:
ds = collect_prediction_data(time_start=t1, time_end=t2, verbose=True)